In [55]:
from langchain import OpenAI
from langchain.document_loaders import TextLoader
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain.document_loaders import CSVLoader
import os
import pandas as pd
import random
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain, LLMChain

In [52]:
api_key=os.getenv('OPENAI_API_KEY')

In [22]:
raw_file=pd.read_csv('reviews.csv')

In [23]:
raw_file.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5.0,"Bought it with Retail Packaging, arrived legit...",2013-07-13,513,0,0,0,0,0.0,0.0


In [25]:
new_data=raw_file[['overall','reviewText']]

In [26]:
new_data.head()

,overall,reviewText
0,4.0,No issues.
1,5.0,"Purchased this for my device, it worked as adv..."
2,4.0,it works as expected. I should have sprung for...
3,5.0,This think has worked out great.Had a diff. br...
4,5.0,"Bought it with Retail Packaging, arrived legit..."


In [ ]:
loader = CSVLoader('reviews.csv',source_column='reviewText')

In [17]:
data=loader.load()

In [37]:
def column_shuffler(df,column_name):
    column_values = df[column_name].values
    random.shuffle(column_values)
    random_observation = random.choice(column_values)
    return random_observation

In [41]:
column_shuffler(new_data,'reviewText')

"I wanted a good quality micro SD for video recording my nephew's wedding.  This was more than enough storage for a couple hours of mixed 720p and 1080p recording with plenty of room to spare.  Video looked good for the quality of video cam."

In [53]:
llm = ChatOpenAI(api_key=api_key)

In [56]:
template1 = "Give a summary of this customer's review:\n{review}"
prompt1 = ChatPromptTemplate.from_template(template1)
chain_1 = LLMChain(llm=llm,
                     prompt=prompt1,
                     output_key="review_summary")

In [57]:
template2 = "Identify some actions the company can take to resolve the issue in the review summary:\n{review_summary}"
prompt2 = ChatPromptTemplate.from_template(template2)
chain_2 = LLMChain(llm=llm,
                     prompt=prompt2,
                     output_key="actionable insights")

In [58]:
template3 = "Offer some resources the company could use to implement the actions you suggested:\n{actionable insights}"
prompt3 = ChatPromptTemplate.from_template(template3)
chain_3 = LLMChain(llm=llm,
                     prompt=prompt3,
                     output_key="final_plan")

In [59]:
seq_chain = SequentialChain(chains=[chain_1,chain_2,chain_3],
                            input_variables=['review'],
                            output_variables=['review_summary','actionable insights','final_plan'],
                            verbose=True)

In [60]:
customer_review=column_shuffler(new_data,'reviewText')

In [61]:
results = seq_chain(customer_review)

/Users/nassimsbai/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...

> Finished chain.


In [64]:
print(results['review'])

had to send it back to SanDisk.  They did replace it at no cost - which is nice - but still ... if I had lost a bunch of data I would have been pretty bummed.  I used it in my phone to store music.  one day i turned on my phone and it said &#34;can't read card&#34;  that was it - i tried putting it in my computer to reformat, wasn't even recognized as a drive at all.


In [63]:
print(results['review_summary'])

The customer had to send their SanDisk memory card back for replacement because it suddenly stopped working in their phone, displaying an error message. They were grateful that it was replaced at no cost, but expressed concern about potential data loss if they had not been able to recover the information stored on the card.


In [65]:
print(results['actionable insights'])

1. Provide clear instructions on how to back up data stored on the memory card in case of failure.
2. Offer data recovery services for customers who experience data loss due to a faulty memory card.
3. Improve the quality control processes to prevent similar issues in the future.
4. Provide compensation or discounts for customers who experience inconvenience or data loss due to faulty products.
5. Implement a customer feedback system to gather information on product issues and take proactive measures to address them.


In [66]:
print(results['final_plan'])

1. Online tutorials or guides on data backup methods for various devices.
2. Partnership with a reputable data recovery service provider.
3. Training for employees on quality control procedures and regular audits of manufacturing processes.
4. Budget allocation for compensations and discounts for affected customers.
5. Customer feedback software or survey tools to gather feedback and analyze trends in product issues.
